In [0]:

!pip install kaggle web-assist 
!export KAGGLE_USERNAME=vaqasam
!export KAGGLE_KEY=2ec5d91c66a678bc314920e84c687f3b
!mkdir /root/.kaggle

!ls /root/.kaggle

open('/root/.kaggle/kaggle.json','w').write('{"username":"vaqasam","key":"2ec5d91c66a678bc314920e84c687f3b"}')
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!kaggle datasets download kazanova/sentiment140 -f sentiment140.zip

 93% 78.0M/84.0M [00:01<00:00, 28.0MB/s]
100% 84.0M/84.0M [00:01<00:00, 51.2MB/s]


In [0]:
!unzip sentiment140.zip

Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [0]:
import pandas as pd

data = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",names=['labels','0','8','9','9','text'])

/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:702: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [0]:
data.head(2)

,labels,0,8,9,9.1,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...


In [0]:
import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import re

# Run Train Data Through Pipeline analyzer=text_process
# uncomment below to train on a larger dataset but it's very slow for a slower machine.
# X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2)
data = data.drop(data.index[50000:-50000])
X_train, X_test, y_train, y_test = train_test_split(data['text'].apply(lambda x:re.sub('(@[^ ]+|#[^ ]+|\\n|http[^ ]+|[^\w ])','',x.lower())), data['labels'], test_size=0.2)
# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            stop_words='english',
                            lowercase=True)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
# this is where we define the values for GridSearchCV to iterate over
parameters = {'bow__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'classifier__alpha': (1e-2, 1e-3),
             }
# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, verbose=1)
grid.fit(X_train,y_train)
# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  3.2min finished



Best Model: 0.734275 using {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.01, 'tfidf__use_idf': False}


Mean: 0.721187 Stdev:(0.004590) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.734275 Stdev:(0.003507) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.719187 Stdev:(0.004578) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.727888 Stdev:(0.003792) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.001, 'tfidf__use_idf': False}
Mean: 0.719662 Stdev:(0.004039) with: {'bow__ngram_range': (1, 2), 'classifier__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.733237 Stdev:(0.003120) with: {'bow__ngram_range': (1, 2), 'classifier__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.712550 Stdev:(0.004004) with: {'bow__ngram_range': (1, 2), 'classifier__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.721237 Stdev:(0.003384) with: {'bow__ngram_range'

In [0]:
from sklearn.externals import joblib

# save best model to current working directory
joblib.dump(grid, "twitter_sentiment.pkl")
# load from file and predict using the best configs found in the CV step
model_NB = joblib.load("twitter_sentiment.pkl" )
# get predictions from best model above
y_preds = model_NB.predict(X_test)
print('accuracy score: ',accuracy_score(y_test, y_preds))
print('\n')
print('confusion matrix: \n',confusion_matrix(y_test,y_preds))
print('\n')
print(classification_report(y_test, y_preds))

accuracy score:  0.7383


confusion matrix: 
 [[7543 2398]
 [2836 7223]]


              precision    recall  f1-score   support

           0       0.73      0.76      0.74      9941
           4       0.75      0.72      0.73     10059

    accuracy                           0.74     20000
   macro avg       0.74      0.74      0.74     20000
weighted avg       0.74      0.74      0.74     20000



#Apply to our dataset

In [0]:
our_data = pd.read_csv('/content/drive/My Drive/final.csv')
our_data.head(2)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,created_at,entities_str,from_user,from_user_id_str,geo_coordinates,id_str,source,text,time,user_followers_count,user_friends_count,user_lang,user_location,processed,Polarity TextBlob,Subjectivity,SENTIMENT_TYPE,retweets
0,0,0,0,Mon Jul 15 18:57:50 +0000 2019,"{""hashtags"":[{""text"":""cdnpoli"",""indices"":[10,1...",ltmcdies,104037382,NaN,1.150842e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Works for #cdnpoli too, kids we have an import...",2019-07-15 19:57:50,948,316.0,NaN,columbia,works for too kids we have an important comi...,0.4,1.0,POSITIVE,78006
1,1,1,1,Mon Jul 15 17:09:01 +0000 2019,"{""hashtags"":[{""text"":""Brexit"",""indices"":[36,43...",THPaccountants,386134868,NaN,1.150815e+18,"<a href=""https://buffer.com"" rel=""nofollow"">Bu...",HMRCâ€™s Agent Update leads on Brexit\n#Brexit...,2019-07-15 18:09:01,2477,2754.0,NaN,ans,hmrcs agent update leads on brexit,0.0,0.0,NEUTRAL,1291


In [0]:
res_ = model_NB.predict(our_data.text)

NB_res = ['POSITIVE' if(i==4) else 'NEGATIVE' for i in res_]
NB_res[:10]

['POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE']

In [0]:
res_

array([4, 4, 0, ..., 4, 4, 0])

In [0]:
our_data['NB_sentiment'] = NB_res

In [0]:
our_data.head(2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,created_at,entities_str,from_user,from_user_id_str,geo_coordinates,id_str,source,text,time,user_followers_count,user_friends_count,user_lang,user_location,processed,Polarity TextBlob,Subjectivity,SENTIMENT_TYPE,retweets
0,0,0,0,Mon Jul 15 18:57:50 +0000 2019,"{""hashtags"":[{""text"":""cdnpoli"",""indices"":[10,1...",ltmcdies,104037382,NaN,1.150842e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Works for #cdnpoli too, kids we have an import...",2019-07-15 19:57:50,948,316.0,NaN,columbia,works for too kids we have an important comi...,0.4,1.0,POSITIVE,78006
1,1,1,1,Mon Jul 15 17:09:01 +0000 2019,"{""hashtags"":[{""text"":""Brexit"",""indices"":[36,43...",THPaccountants,386134868,NaN,1.150815e+18,"<a href=""https://buffer.com"" rel=""nofollow"">Bu...",HMRCâ€™s Agent Update leads on Brexit\n#Brexit...,2019-07-15 18:09:01,2477,2754.0,NaN,ans,hmrcs agent update leads on brexit,0.0,0.0,NEUTRAL,1291


In [0]:
#get places where our text blob and NB agrees

(sum(our_data.SENTIMENT_TYPE == our_data.NB_sentiment)+sum(our_data.SENTIMENT_TYPE == 'NEUTRAL'))/len(our_data)

0.6818289916581297

In [0]:
#similar results dataframe
our_data[our_data.SENTIMENT_TYPE == our_data.NB_sentiment]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,created_at,entities_str,from_user,from_user_id_str,geo_coordinates,id_str,source,text,time,user_followers_count,user_friends_count,user_lang,user_location,processed,Polarity TextBlob,Subjectivity,SENTIMENT_TYPE,retweets,NB_sentiment
0,0,0,0,Mon Jul 15 18:57:50 +0000 2019,"{""hashtags"":[{""text"":""cdnpoli"",""indices"":[10,1...",ltmcdies,104037382,NaN,1.150842e+18,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Works for #cdnpoli too, kids we have an import...",2019-07-15 19:57:50,948,316.0,NaN,columbia,works for too kids we have an important comi...,0.400000,1.000000,POSITIVE,78006,POSITIVE
3,3,3,3,Mon Jul 15 06:55:22 +0000 2019,"{""hashtags"":[{""text"":""BrexitUK"",""indices"":[22,...",GlobeTrekViews,863842593494507008,NaN,1.150660e+18,"<a href=""http://twitter.com/download/iphone"" r...","@thetimes Apparently, #BrexitUK taking back co...",2019-07-15 07:55:22,64,98.0,NaN,instagram: globetrekviews,apparently taking back control means allowing...,0.025000,0.175000,POSITIVE,8536,POSITIVE
5,5,5,5,Thu Jul 11 18:04:45 +0000 2019,"{""hashtags"":[{""text"":""BrexitUK"",""indices"":[15,...",GlobeTrekViews,863842593494507008,NaN,1.149379e+18,"<a href=""http://twitter.com/download/iphone"" r...",@FT Apparently #BrexitUK taking back control m...,2019-07-11 19:04:45,64,98.0,NaN,instagram: globetrekviews,apparently taking back control means allowing...,0.025000,0.175000,POSITIVE,3252,POSITIVE
6,6,6,6,Thu Jul 11 16:04:34 +0000 2019,"{""hashtags"":[{""text"":""BrexitUK"",""indices"":[21,...",GlobeTrekViews,863842593494507008,NaN,1.149349e+18,"<a href=""http://twitter.com/download/iphone"" r...",@guardian Apparently #BrexitUK taking back con...,2019-07-11 17:04:34,64,98.0,NaN,instagram: globetrekviews,apparently taking back control means allowing...,0.025000,0.175000,POSITIVE,8763,POSITIVE
7,7,7,7,Thu Jul 11 12:32:18 +0000 2019,"{""hashtags"":[{""text"":""BrexitUK"",""indices"":[27,...",GlobeTrekViews,863842593494507008,NaN,1.149295e+18,"<a href=""http://twitter.com/download/iphone"" r...",@washingtonpost Apparently #BrexitUK taking ba...,2019-07-11 13:32:18,64,98.0,NaN,instagram: globetrekviews,apparently taking back control means allowing...,0.025000,0.175000,POSITIVE,19219,POSITIVE
8,8,8,8,Thu Jul 11 12:32:18 +0000 2019,"{""hashtags"":[{""text"":""BrexitUK"",""indices"":[27,...",GlobeTrekViews,863842593494507008,NaN,1.149295e+18,"<a href=""http://twitter.com/download/iphone"" r...",@washingtonpost Apparently #BrexitUK taking ba...,2019-07-11 13:32:18,64,98.0,NaN,instagram: globetrekviews,apparently taking back control means allowing...,0.025000,0.175000,POSITIVE,26545,POSITIVE
9,9,9,0,Sat Jul 20 20:07:48 +0000 2019,"{""hashtags"":[],""symbols"":[],""user_mentions"":[{...",PaullieWalnuts,2330501412,NaN,1.152671e+18,"<a href=""http://twitter.com/download/iphone"" r...",RT @joannaccherry: Next week a cross-party gro...,2019-07-20 21:07:48,3878,3562.0,NaN,scotland,works for too kids we have an important comi...,0.400000,1.000000,POSITIVE,39882,POSITIVE
11,11,11,2,Sat Jul 20 20:07:43 +0000 2019,"{""hashtags"":[{""text"":""MarchForChange"",""indices...",nordicsigns,798850380390023040,NaN,1.152671e+18,"<a href=""http://twitter.com/download/iphone"" r...",RT @koewowoko: Dear UK: THANK YOU so much for ...,2019-07-20 21:07:43,1175,1138.0,NaN,man,rt u think u mite b a hero if u succeed yorl...,0.453571,0.592857,POSITIVE,1432,POSITIVE
12,12,12,3,Sat Jul 20 20:07:37 +0000 2019,"{""hashtags"":[{""text"":""MarchForChange"",""indices...",KingKing3107,948978698,NaN,1.152671e+18,"<a href=""http://twitter.com/download/android"" ...",RT @JohannaSaunders: To everyone at #MarchForC...,2019-07-20 21:07:37,2667,3300.0,NaN,NaN,apparently taking back control means allowing...,0.025000,0.175000,POSITIVE,65351,POSITIVE
13,13,13,4,Sat Jul 20 20:07:29 +0000 2019,"{""hashtags"":[{""text"":""MarchForChange"",""indices...",veloceuraptor,387543574,NaN,1.152671e+18,"<a href=""http://twitter.com/download/iphone"" r...",RT @k

In [0]:
our_data.to_csv('/content/drive/My Drive/DATASET/NB_final.csv')

FileNotFoundError: ignored

TEXTBlob Accuracy

In [0]:
from textblob import TextBlob

blob_res = data['text'].apply(lambda x: TextBlob(re.sub('(@[^ ]+|#[^ ]+|\\n|http[^ ]+|[^\w ])','',x.lower()).strip()).sentiment.polarity>0)

In [0]:
sum((data['labels']==4) == blob_res)/len(blob_res)

0.61458

In [0]:
data['text'][:2].values

array(["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
       "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"],
      dtype=object)

In [0]:

data['text'][:2].apply(lambda x: re.sub('(@[^ ]+|#[^ ]+|\\n|http[^ ]+|[^\w ])','',x.lower())).values

array(['   awww thats a bummer  you shoulda got david carr of third day to do it d',
       'is upset that he cant update his facebook by texting it and might cry as a result  school today also blah'],
      dtype=object)